In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 16.4 MB/s eta 0:00:00


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import urllib
import zipfile
import random
import tensorflow as tf
import tensorflow.keras.layers.experimental.preprocessing as augment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [2]:

def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/satellitehurricaneimages.zip'
    urllib.request.urlretrieve(url, 'satellitehurricaneimages.zip')
    with zipfile.ZipFile('satellitehurricaneimages.zip', 'r') as zip_ref:
        zip_ref.extractall()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


callbacks = [
        # EarlyStopping(
        #     monitor='val_accuracy',
        #     min_delta=1e-4,
        #     patience=3,
        #     verbose=1
        # )
      #  ,
        ModelCheckpoint(
            filepath='mymodel.h5',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        )
        # ,

        # ReduceLROnPlateau(monitor="val_loss",
        #                                         factor=0.2,  # multiply the learning rate by 0.2 (reduce by 5x)
        #                                         patience=2,
        #                                         verbose=1,  # print out when learning rate goes down
        #                                         min_lr=1e-7)
        ]



In [3]:

download_and_extract_data()
IMG_SIZE = (128, 128)
BATCH_SIZE = 64

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  directory='train/',
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE,
  label_mode = 'binary')

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  directory='validation/',
  image_size= IMG_SIZE,
  batch_size=BATCH_SIZE,
  label_mode = 'binary')

Found 10000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [4]:
def preprocess(image, label):
    image = image/255
    label = label/255




    return image, label



In [5]:
train_ds = train_ds.map(
preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
tf.data.experimental.AUTOTUNE)
val_ds = val_ds.map(
preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds, val_ds




(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>,
 <_ParallelMapDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>)

In [6]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.layers.experimental.preprocessing as augment

# # Create a data augmentation stage with horizontal flipping, rotations, zooms
# data_augmentation = keras.Sequential(
#     [
#         layers.RandomFlip("horizontal"),
#         layers.RandomRotation(0.1),
#         layers.RandomZoom(0.1),
#     ]
# )

data_augmentation = tf.keras.Sequential([
  augment.RandomFlip("horizontal_and_vertical"),
  augment.RandomRotation(0.2),
  augment.RandomZoom(height_factor=(0.2, 0.3), width_factor=(0.2, 0.3)),
  augment.RandomTranslation(0.3, 0.3, fill_mode='reflect', interpolation='bilinear',)
])

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1 / 255)

train_generator = train_datagen.flow_from_directory(
    'train/',  # This is the source directory for training images
    target_size=(300, 300),  # All images will be resized to 150x150
    batch_size=1,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    'validation/',  # This is the source directory for training images
    target_size=(300, 300),  # All images will be resized to 150x150
    batch_size=1,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            min_delta=1e-4,
            patience=3,
            verbose=1
        ),
        ModelCheckpoint(
            filepath='mymodel.h5',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        )
    ]

In [ ]:
train_ds, val_ds

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
 <_ParallelMapDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>)

In [ ]:
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop


#base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')



weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

pre_trained_model = InceptionV3(input_shape=(128, 128, 3),
                                include_top=False,
                                weights=None)

pre_trained_model.load_weights(weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1 / 255)

train_generator = train_datagen.flow_from_directory(
    'train/',  # This is the source directory for training images
    target_size=(128, 128),  # All images will be resized to 150x150
    batch_size=64,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    'validation/',  # This is the source directory for training images
    target_size=(128, 128),  # All images will be resized to 150x150
    batch_size=64,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary')





model.fit(
        train_generator,
        steps_per_epoch=8,
        epochs=100,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=8,
        callbacks=callbacks
    )


NameError: ignored

In [ ]:
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.optimizers import RMSprop







# base_model = EfficientNetB7(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet', drop_connect_rate=0.4)

# for layer in base_model.layers:
#     layer.trainable = False



# model = tf.keras.Sequential([
#                                 base_model,
#                                 tf.keras.layers.GlobalAveragePooling2D(name="Layer1"),
#                                 tf.keras.layers.Dropout(0.4),
#                                 tf.keras.layers.Dense(1, activation='softmax')
#     ])
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='binary_crossentropy',metrics=['accuracy'], run_eagerly=True)



# train_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

# validation_datagen = ImageDataGenerator(rescale=1 / 255)

# train_generator = train_datagen.flow_from_directory(
#     'train/',  # This is the source directory for training images
#     target_size=(128, 128),  # All images will be resized to 150x150
#     batch_size=64,
#     # Since we use binary_crossentropy loss, we need binary labels
#     class_mode='binary')

# validation_generator = validation_datagen.flow_from_directory(
#     'validation/',  # This is the source directory for training images
#     target_size=(128, 128),  # All images will be resized to 150x150
#     batch_size=64,
#     # Since we use binary_crossentropy loss, we need binary labels
#     class_mode='binary')


#train_ds, val_ds





In [ ]:
model.fit(
        train_ds,
        steps_per_epoch=100,
        epochs=10,
        verbose=1,
        validation_data=val_ds,
        validation_steps=8,
        callbacks=callbacks
    )

In [ ]:
model = keras.Sequential([
    layers.Input((128,128,3)),

    # Making the base
    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),

    # Making the head
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid') # used sigmoid as activation function because we are using binary labels
])

OPTIMIZER = keras.optimizers.Adam(
    learning_rate=1e-4
)
LOSS = keras.losses.binary_crossentropy
model.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=['binary_accuracy']
)

model.summary()

early_stopping = EarlyStopping(
    min_delta=1e-3,
    patience=4,
    verbose=1,
    restore_best_weights=True
)

ckpt = ModelCheckpoint(
    'model_weights/hurricane-model-weights.hdf5',
    verbose=1,
    save_best_only=True,
    save_weights_only=True
)

history = model.fit(
    train_ds,
    validation_data=(val_ds),
    batch_size=64,
    callbacks=[early_stopping, ckpt],
    verbose=1,
    epochs=200
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model

def create_model():
  base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
  # for layer in base_model.layers[:-30]:
  #   layer.trainable = False
  augmented = data_augmentation(base_model)
  resnet = base_model(augmented)
  pooling = layers.GlobalAveragePooling2D()(resnet)
  dropout = layers.Dropout(0.4)(pooling)
  outputs = Dense(len(class_types), activation="softmax")(dropout)
  model = Model(inputs=inputs, outputs=outputs)

  return model

model = create_model()
model.summary()

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
              metrics=['acc'])



history = model.fit(train_ds,
                    epochs = 60,
                    validation_data=val_ds,
                    callbacks=callbacks)

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Dropout

# Initialize a sequential model
model = Sequential(name="Base_Model")


model.add(Conv2D(32,kernel_size =(3, 3), activation='relu',input_shape=(128,128,3)))
model.add(data_augmentation)
model.add(Conv2D(32,kernel_size =(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64,kernel_size =(3,3), activation='relu'))
model.add(Conv2D(64,kernel_size =(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128,kernel_size =(3,3), activation='relu'))
model.add(Conv2D(128,kernel_size =(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(1024,activation='relu'))
model.add(Dense(1024,activation='relu'))

model.add(Dense(1,activation = 'sigmoid'))

In [ ]:
model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate = 1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_1 = model.fit(train_ds,validation_data=val_ds,epochs=10)